# A mathematical model of photoinhibition: exploring the impact of quenching processes

- Tim Nies <a href="https://orcid.org/0000-0003-1587-2971"><img src="https://orcid.org/assets/vectors/orcid.logo.icon.svg" width=20></a>
- Shizue Matsubara <a href="https://orcid.org/0000-0002-1440-6496"><img src="https://orcid.org/assets/vectors/orcid.logo.icon.svg" width=20></a>
- Oliver Ebenhöh <a href="https://orcid.org/0000-0002-7229-7398"><img src="https://orcid.org/assets/vectors/orcid.logo.icon.svg" width=20></a>

bioarxiv: 

### Macros

In [ ]:
FIG_DIR = "reports/figures/"
RES_DIR = "reports/results/"
SPINT = 10_000
HL = 800
DARK = 0.001
LANDSCAPE_SIZE = (15, 7.5)
DPI = 400
TIFF = False


### Load packages

In [ ]:
from copy import deepcopy
from math import atan2, degrees

import matplotlib.patheffects as mpe
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np
import os
import pandas as pd
import pickle
from modelbase.ode import Simulator
from scipy.interpolate import make_interp_spline

from src.model_v2 import model
from src.utils import changingLight, FvFm_experiment2

#### Helper functions

In [ ]:
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created.")
    else:
        print(f"Directory '{directory_path}' already exists.")


def check_file_exists(file_path):
    if os.path.exists(file_path):
        return True
    else:
        False

### Create missing directories

In [ ]:
create_directory(FIG_DIR)
create_directory(RES_DIR)

### Parameters for fluorescence yield models

In [ ]:
kH = model.get_parameter("kH")
kP = model.get_parameter("kP")
kF = model.get_parameter("kF")
Q = 0.1
r = model.get_parameter("beta")
PSII = model.get_parameter("PSIItot")
kT = model.get_parameter("kT")  # electron transfer
kT2 = 8e8  # back and forth electron transfer

### Figure 2: Fluorescence yield model 

In [ ]:
def FvFmi(Ua, kF, kH, kP, Q, PSII, r):
    a = kP / (kF + kH * Q + kP)
    b = (kH * Q * r + kF) / (Ua * kH * Q * (r - 1) + PSII * (kH * Q + kF))
    return Ua * a * b


def Fm(Ua, kF, kH, kP, Q, PSII, r):
    Fm = kF / (kF + kH * Q) * Ua + kF / (kF + kH * Q * r) * (PSII - Ua)
    return Fm


def Fo(Ua, kF, kH, kP, Q, PSII, r):
    Fo = kF / (kF + kH * Q + kP) * Ua + kF / (kF + kH * Q * r) * (PSII - Ua)
    return Fo


def Fo_slope(r, Q, kF, kP, kH):
    num = kF * (Q * kH * (r - 1) - kP)
    denom = (Q * kH * r + kF) * (Q * kH + kF + kP)
    return num / denom


def Fm_slope(r, Q, kF, kP, kH):
    num = Q * kF * kH * (r - 1)
    denom = (Q * kH + kF) * (Q * kH * r + kF)
    return num / denom


def Fo_rel_slope(r, Q, kF, kP, kH):
    num = Q * kH * (r - 1) - kP
    denom = PSII * (Q * kH * r + kF)
    return num / denom


def Fm_rel_slope(r, Q, kF, kP, kH):
    num = Q * kH * (r - 1)
    denom = PSII * (Q * kH * r + kF)
    return num / denom


def slope_ratio(r, Q, kH, kF, kP):
    num = (Q * kH + kF) * (Q * kH * (r - 1) - kP)
    denom = Q * kH * (r - 1) * (Q * kH + kF + kP)
    return num / denom


def slope_ratio_rel(r, Q, kH, kF, kP):
    num = Q * kH * (r - 1) - kP
    denom = Q * kH * (r - 1)
    return num / denom


def rmone(Q, kH, kF, kP):
    num = 2 * Q**2 * kH**2 + 2 * Q * kF * kH + 2 * Q * kH * kP + kF * kP
    denom = Q * kH * (2 * Q * kH + 2 * kF + kP)
    return num / denom


def rmone_rel(Q, kH, kP):
    r = kP / (2 * kH * Q) + 1
    return r


def rzero(Q, kH, kP):
    r = kP / (kH * Q) + 1
    return r


def Fo_rel(Ua, kF, kH, kP, Q, PSII, r):
    num = Ua * (Q * kH * r + kF) + (PSII - Ua) * (Q * kH + kF + kP)
    denom = PSII * (Q * kH * r + kF)
    return num / denom


def Fm_rel(Ua, kF, kH, kP, Q, PSII, r):
    num = Ua * (Q * kH * r + kF) + (PSII - Ua) * (Q * kH + kF)
    denom = PSII * (Q * kH * r + kF)
    return num / denom

In [ ]:
Ua_range = np.linspace(0, 2.5, 100)

r_range = [0.1, 1, 2]

fig, ax = plt.subplots(1, 3, figsize=(21, 7))
for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linewidth=3,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle="--",
        )


ax[0].grid()
ax[0].annotate(r"$\rho=2$", (0.5, 0.7), size=18)
ax[0].annotate(r"$\rho=0.1$", (0.2, 0.20), size=18)
ax[0].set_xlim(0, 1)
ax[0].set_ylim(0, 1)
ax[0].set_xlabel("inactive PSII fraction", size=18)
ax[0].set_ylabel(r"F$\mathrm{_v}$/F$\mathrm{_m}$, relative units", size=18)
ax[0].vlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].hlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r) / Fm(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r) / Fm(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r) / Fm(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle="--",
        )


for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle="--",
        )

ax[1].grid()
ax[1].annotate(r"$\rho=2$", (0.2, 0.6), size=18)
ax[1].annotate(r"$\rho=0.1$", (0.35, 3), size=18)
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 4)
ax[1].set_xlabel("inactive PSII fraction", size=18)
ax[1].set_ylabel(r"F$\mathrm{_m}$, relative units", size=18)
ax[1].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r) / Fo(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r) / Fo(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r) / Fo(2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle="--",
        )

ax[2].annotate(r"$\rho=2$", (0.2, 0.7), size=18)
ax[2].annotate(r"$\rho=0.1$", (0.08, 3), size=18)
ax[2].grid()
ax[2].set_xlim(0, 1)
ax[2].set_ylim(0, 4)
ax[2].set_xlabel("inactive PSII fraction", size=18)
ax[2].set_ylabel(r"F$\mathrm{_o}$, relative units", size=18)
ax[2].tick_params("both", labelsize=18)
plt.tight_layout()
if TIFF:
    plt.savefig(FIG_DIR + "FvFm_inh.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "FvFm_inh.png", dpi=DPI)
plt.show()

### Figure 3

In [ ]:
r_range2 = np.linspace(0, 12, 1000)
sr = slope_ratio_rel(r_range2, Q, kH, kF, kP)
rm = rmone_rel(Q, kH, kP)
rz = rzero(Q, kH, kP)

srQ = slope_ratio_rel(r_range2, Q * 10, kH, kF, kP)
rmQ = rmone_rel(Q * 10, kH, kP)
rzQ = rzero(Q * 10, kH, kP)

plt.plot(r_range2, sr, linewidth=3, c="k")
plt.plot(r_range2, srQ, linewidth=3, c="k", linestyle="dotted")
plt.vlines(rm, -6, 1, color="r")
plt.vlines(rz, -6, 1, color="orange")
plt.vlines(rmQ, -6, 1, color="r", linestyle="dotted", linewidth=2)
plt.vlines(rzQ, -6, 1, color="orange", linestyle="dotted", linewidth=2)
plt.scatter(rm, -1, color="r", s=50)
plt.scatter(rz, 0, color="orange", s=50)
plt.scatter(rmQ, -1, color="r", s=50)
plt.scatter(rzQ, 0, color="orange", s=50)
plt.grid()
plt.xlim(1, 12)
plt.ylim(-6, 1)
plt.xlabel(r"$\rho$", size=15)
plt.ylabel(r"slope ratio ($\gamma$), relative units", size=15)
plt.tick_params("both", labelsize=15)
plt.tight_layout()
if TIFF:
    plt.savefig(FIG_DIR + "slope_ratio_rel.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "slope_ratio_rel.png", dpi=DPI)
plt.show()

### Figure 4: Fluorescence yield model with electron transfer

In [ ]:
def Fo(Ua, kF, kH, kP, Q, PSII, r, kT):
    return kF / (kF + kH * Q + kP) * Ua + kF / (kF + kH * Q * r) * (PSII - Ua)


def Fm(Ua, kF, kH, kP, Q, PSII, r, kT):
    return kF / (kF + kH * Q + kT * (PSII - Ua)) * Ua + kF / (kF + kH * Q * r) * (
        PSII - Ua
    )


def FvFmi(Ua, kF, kH, kP, Q, PSII, r, kT):
    FvFm = kP / (kP + kH * Q + kF)
    a = kT * (PSII - Ua) / (kF + kH * Q + kP)
    num = kH * Q * r + kF
    denom = Ua * (kF + kH * Q * r) + (PSII - Ua) * (kF + kH * Q + kT * (PSII - Ua))
    b = num / denom

    return b * Ua * (FvFm - a)


def extremumFm(kF, kH, kP, Q, PSII, r, kT):
    a = PSII * kT + Q * kH + kF
    b = (Q * kH * r + kF) * (PSII * kT + Q * kH + kF)
    ex1 = (a - np.sqrt(b)) / kT
    ex2 = (a + np.sqrt(b)) / kT
    return ex1

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(21, 7))
for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linewidth=3,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linestyle="--",
        )


ax[0].grid()
ax[0].annotate(r"$\rho=2$", (0.5, 0.55), size=18)
ax[0].annotate(r"$\rho=0.1$", (0.1, 0.20), size=18)
ax[0].set_xlim(0, 1)
ax[0].set_ylim(0, 1)
ax[0].set_xlabel("inactive PSII fraction", size=18)
ax[0].set_ylabel(r"F$\mathrm{_v}$/F$\mathrm{_m}$, relative units", size=18)
ax[0].vlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].hlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / Fm(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / Fm(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / Fm(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linestyle="--",
        )


for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linestyle="--",
        )

ax[1].grid()
ax[1].annotate(r"$\rho=2$", (0.2, 0.4), size=18)
ax[1].annotate(r"$\rho=0.1$", (0.3, 3), size=18)
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 4)
ax[1].set_xlabel("inactive PSII fraction", size=18)
ax[1].set_ylabel(r"F$\mathrm{_m}$, relative units", size=18)
ax[1].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / Fo(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / Fo(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r, kT)
            / Fo(2.5, kF, kH, kP, Q, PSII, r, kT),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r, kT),
            c="r",
            linestyle="--",
        )

ax[2].annotate(r"$\rho=2$", (0.2, 0.8), size=18)
ax[2].annotate(r"$\rho=0.1$", (0.08, 3), size=18)
ax[2].grid()
ax[2].set_xlim(0, 1)
ax[2].set_ylim(0, 4)
ax[2].set_xlabel("inactive PSII fraction", size=18)
ax[2].set_ylabel(r"F$\mathrm{_o}$, relative units", size=18)
ax[2].tick_params("both", labelsize=18)
plt.tight_layout()
if TIFF:
    plt.savefig(FIG_DIR + "FvFm_inhAET.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "FvFm_inhAET.png", dpi=DPI)
plt.show()

In [ ]:
Q_range = np.linspace(0.1, 1, 20)
r_range3 = np.linspace(0.705, 1.4, 20)

Q_mesh, r_mesh = np.meshgrid(Q_range, r_range3)
E_mesh = extremumFm(kF, kH, kP, Q_mesh, PSII, r_mesh, kT) / 2.5

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
surf = ax.plot_surface(
    Q_mesh,
    r_mesh,
    E_mesh,
    cmap=cm.coolwarm,
    linewidth=0.025,
    antialiased=False,
    edgecolors="grey",
)

# Customize the z axis.
# ax.set_zlim(-1,1)
ax.zaxis.set_major_locator(LinearLocator(10))
# A StrMethodFormatter is used automatically
ax.zaxis.set_major_formatter("{x:.02f}")

# Add a color bar which maps values to colors.
# fig.colorbar(surf, shrink=0.5, aspect=5)
ax.view_init(20, 50)
plt.xlabel("Quencher activity")
plt.ylabel(r"$\rho$")
ax.set_zlabel("active PSII fraction")
if TIFF:
    plt.savefig(FIG_DIR + "Fmmax.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "Fmmax.png", dpi=DPI)
plt.show()

### Figure S8: Fluorescence yield model with back and forth energy transfer

In [ ]:
def Fo(Ua, kF, kH, kP, Q, PSII, r, kT2):
    return kF / (kF + kH * Q + kP) * Ua + kF / (kF + kH * Q * r + kT2 * Ua) * (
        PSII - Ua
    )


def Fm(Ua, kF, kH, kP, Q, PSII, r, kT2):
    return kF / (kF + kH * Q + kT2 * (PSII - Ua)) * Ua + kF / (
        kF + kH * Q * r + kT2 * Ua
    ) * (PSII - Ua)


def FvFmi(Ua, kF, kH, kP, Q, PSII, r, kT2):
    FvFm = kP / (kP + kH * Q + kF)
    a = kT2 * (PSII - Ua) / (kF + kH * Q + kP)
    num = kH * Q * r + Ua * kT2 + kF
    denom = Ua * (kF + kH * Q * r + kT2 * Ua) + (PSII - Ua) * (
        kF + kH * Q + kT2 * (PSII - Ua)
    )
    b = num / denom

    return b * Ua * (FvFm - a)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(21, 7))
for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / FvFmi(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linewidth=3,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / FvFmi(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linestyle="--",
        )


ax[0].grid()
ax[0].annotate(r"$\rho=2$", (0.55, 0.55), size=18)
ax[0].annotate(r"$\rho=0.1$", (0.3, 0.20), size=18)
ax[0].set_xlim(0, 1)
ax[0].set_ylim(0, 1)
ax[0].set_xlabel("inactive PSII fraction", size=18)
ax[0].set_ylabel(r"F$\mathrm{_v}$/F$\mathrm{_m}$, relative units", size=18)
ax[0].vlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].hlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / Fm(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / Fm(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / Fm(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linestyle="--",
        )


for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / Fm(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linestyle="--",
        )

ax[1].grid()
ax[1].annotate(r"$\rho=2$", (0.8, 0.4), size=18)
ax[1].annotate(r"$\rho=0.1$", (0.6, 3), size=18)
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 4)
ax[1].set_xlabel("inactive PSII fraction", size=18)
ax[1].set_ylabel(r"F$\mathrm{_m}$, relative units", size=18)
ax[1].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / Fo(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / Fo(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q, PSII, r, kT2)
            / Fo(2.5, kF, kH, kP, Q, PSII, r, kT2),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(Ua_range, kF, kH, kP, Q * 10, PSII, r, kT2)
            / Fo(2.5, kF, kH, kP, Q * 10, PSII, r, kT2),
            c="r",
            linestyle="--",
        )

ax[2].annotate(r"$\rho=2$", (0.2, 0.8), size=18)
ax[2].annotate(r"$\rho=0.1$", (0.3, 3), size=18)
ax[2].grid()
ax[2].set_xlim(0, 1)
ax[2].set_ylim(0, 4)
ax[2].set_xlabel("inactive PSII fraction", size=18)
ax[2].set_ylabel(r"F$\mathrm{_o}$, relative units", size=18)
ax[2].tick_params("both", labelsize=18)
plt.tight_layout()
if TIFF:
    plt.savefig(FIG_DIR + "FvFm_inhET.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "FvFm_inhET.png", dpi=DPI)
plt.show()

### Fig S9: Fluorescence model with PSII heterogenity

In [ ]:
def Fm(a, Ua, kF, kH, kP, Q, PSII, r):
    Fm = a * (kF / (kF + kH * Q) * Ua + kF / (kF + kH * Q * r) * (PSII - Ua)) + (
        1 - a
    ) * PSII * (kF / (kF + kH * Q))
    return Fm


def Fo(a, Ua, kF, kH, kP, Q, PSII, r):
    Fo = a * (kF / (kF + kH * Q + kP) * Ua + kF / (kF + kH * Q * r) * (PSII - Ua)) + (
        1 - a
    ) * PSII * (kF / (kF + kH * Q + kP))
    return Fo


def FvFmi(a, Ua, kF, kH, kP, Q, PSII, r):
    fm = Fm(a, Ua, kF, kH, kP, Q, PSII, r)
    fo = Fo(a, Ua, kF, kH, kP, Q, PSII, r)
    return (fm - fo) / fm


In [ ]:
a = 0.8  # PSIIa fraction

fig, ax = plt.subplots(1, 3, figsize=(21, 7))
for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / FvFmi(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / FvFmi(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / FvFmi(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / FvFmi(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linewidth=3,
        )
    elif r == 2:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / FvFmi(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle=":",
        )
    else:
        ax[0].plot(
            1 - Ua_range / 2.5,
            FvFmi(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / FvFmi(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle="--",
        )


ax[0].grid()
ax[0].annotate(r"$\rho=2$", (0.4, 0.85), size=18)
ax[0].annotate(r"$\rho=0.1$", (0.4, 0.18), size=18)
ax[0].set_xlim(0, 1)
ax[0].set_ylim(0, 1)
ax[0].set_xlabel("inactive PSII fraction", size=18)
ax[0].set_ylabel(r"F$\mathrm{_v}$/F$\mathrm{_m}$, relative units", size=18)
ax[0].vlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].hlines(0.5, 0, 1, color="grey", alpha=0.8)
ax[0].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / Fm(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / Fm(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / Fm(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle="--",
        )


for r in r_range:
    if r == 1:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fm(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fm(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle=":",
        )
    else:
        ax[1].plot(
            1 - Ua_range / 2.5,
            Fm(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fm(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle="--",
        )

ax[1].grid()
ax[1].annotate(r"$\rho=2$", (0.2, 0.5), size=18)
ax[1].annotate(r"$\rho=0.1$", (0.6, 3.5), size=18)
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 4)
ax[1].set_xlabel("inactive PSII fraction", size=18)
ax[1].set_ylabel(r"F$\mathrm{_m}$, relative units", size=18)
ax[1].tick_params("both", labelsize=18)

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / Fo(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / Fo(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(a, Ua_range, kF, kH, kP, Q, PSII, r)
            / Fo(a, 2.5, kF, kH, kP, Q, PSII, r),
            c="k",
            linestyle="--",
        )

for r in r_range:
    if r == 1:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fo(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linewidth=4,
        )
    elif r == 2:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fo(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle=":",
        )
    else:
        ax[2].plot(
            1 - Ua_range / 2.5,
            Fo(a, Ua_range, kF, kH, kP, Q * 10, PSII, r)
            / Fo(a, 2.5, kF, kH, kP, Q * 10, PSII, r),
            c="r",
            linestyle="--",
        )

ax[2].annotate(r"$\rho=2$", (0.2, 0.8), size=18)
ax[2].annotate(r"$\rho=0.1$", (0.1, 3), size=18)
ax[2].grid()
ax[2].set_xlim(0, 1)
ax[2].set_ylim(0, 4)
ax[2].set_xlabel("inactive PSII fraction", size=18)
ax[2].set_ylabel(r"F$\mathrm{_o}$, relative units", size=18)
ax[2].tick_params("both", labelsize=18)
plt.tight_layout()
if TIFF:
    plt.savefig(FIG_DIR + "FvFm_het.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "FvFm_het.png", dpi=DPI)
plt.show()

### Initial conditions

In [ ]:
# initial conditions
y0d = {
    "P": 0,
    "H": 6.339572769844456e-05,
    "A": 0,
    "V": 1.0,
    "Pr": 1.0,
    "E": 0,
    "PC": 1e-10,
    "Fd": 1e-10,
    "NADPH": 12.5,
    "Uinh": 0,
    "Udeg": 0,
    "Uact": 2.5,
}

model.update_parameter("PFD", DARK)
s = Simulator(model)
s.initialise(y0d)
s.simulate_to_steady_state()
ss_o = s.get_results_dict()
for k, v in ss_o.items():
    ss_o[k] = float(v)

#model.update_parameter("kREP",0)
#CL = changingLight(model,ss_o,[DARK,HL,DARK],[20*60,12*60*60])
# CL.plot_selection("Q")
# CL.plot_selection("pH")
# CL.plot_selection("P")
# CL.plot_selection("E")
# CL.plot_selection("Pnr")
# CL.plot_selection("Z")
# CL.plot_selection("A")
# CL.plot_selection("pH")
# CL.plot_flux_selection("mito")
#CL.plot_selection("Uact")
#plt.show()

# model2 = deepcopy(model)
# model2.update_parameter("activationATP_switch",False)
# CL = changingLight(model2,ss_o,[0,5000,0,500,5000,500,5000,0,5000,0,5000,0,5000],[60,0.8,60,60,0.8,5*60,0.8,60,0.8,60,0.8,60,0.8])
# CL.plot_selection("pH")
# CL.plot_selection("Fluo")
# plt.show()

### Fig S5-S7 Check energy transfer

In [ ]:
from src.utils import FvFm_experiment
import scipy


def monoExp(x, m, r):
    return m * np.exp(-r * x)


scenarios = {"ET": {"kREP": 0}, "woET": {"kREP": 0, "kT": 0}}

if check_file_exists(RES_DIR + "ET.p"):
    with open(RES_DIR + "ET.p", "rb") as file:
        res_dict = pickle.load(file)
else:
    res_dict = {}
    for k, v in scenarios.items():
        model_ET = deepcopy(model)
        model_ET.update_parameters(v)
        CL = changingLight(model_ET, ss_o, [HL], [20 * 60 * 60])
        (
            FvFm_lst,
            Fm_lst,
            Fo_lst,
            total_tm,
            PAM1_lst,
            Ua_lst,
            abs_time,
        ) = FvFm_experiment(model_ET, ss_o, SPint=SPINT)
        sce_dict = {}
        sce_dict["t"] = CL.get_time()
        sce_dict["Uact"] = CL.get_variable("Uact")
        sce_dict["B1"] = CL.get_variable("B1")
        sce_dict["B3"] = CL.get_variable("B3")
        sce_dict["Q"] = CL.get_variable("Q")
        sce_dict["P"] = CL.get_variable("P")
        sce_dict["abs_time"] = abs_time
        sce_dict["FvFm"] = FvFm_lst
        sce_dict["Ua_lst"] = Ua_lst

        res_dict[k] = sce_dict

    with open(RES_DIR + "ET.p", "wb") as file:
        pickle.dump(res_dict, file)


fig2, ax2 = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 5))
left, bottom, width, height = [0.22, 0.6, 0.25, 0.25]
ax3 = fig2.add_axes([left, bottom, width, height])
left, bottom, width, height = [0.73, 0.6, 0.25, 0.25]
ax4 = fig2.add_axes([left, bottom, width, height])
fig, ax = plt.subplots(1, 3, constrained_layout=True, figsize=(17, 5))

for k, v in res_dict.items():
    t = v["t"]
    Uact = v["Uact"]
    B1 = v["B1"]
    B3 = v["B3"]
    Q = v["Q"]
    P = v["P"]
    abs_time = v["abs_time"]
    FvFm = v["FvFm"]

    p0 = (
        2.5,
        0.00001,
    )  # start with values near those we expect
    params, cv = scipy.optimize.curve_fit(monoExp, t, Uact, p0)
    m, r = params

    if k == "ET":
        lab = k
        c = "r"
    else:
        lab = "w/o ET"
        c = "orange"

    ax3.plot(t / 3600, Uact / 2.5, color=c)
    ax4.plot(abs_time, FvFm / FvFm[0], color=c)
    ax2[1].plot(abs_time, FvFm / FvFm[0], label=lab, linewidth=3, color=c)
    ax2[0].plot(t / 3600, Uact / 2.5, label=lab, linewidth=3, color=c)
    ax2[0].plot(t / 3600, monoExp(t, m, r) / 2.5, color="k", linestyle=":", linewidth=2)
    ax[0].plot(t / 3600, (B1 + B3) / Uact, label=lab, linewidth=3, color=c)
    ax[1].plot(t / 3600, Q, label=lab, linewidth=3, color=c)
    ax[2].plot(t / 3600, P/model.get_parameter("PQtot"), label=lab, linewidth=3, color=c)

ax2[0].set_xlabel("Time / [h]", fontsize=15)
ax2[0].set_ylabel(r"act. Photosystem II [rel. units]", fontsize=15)
ax2[0].grid()
ax2[0].tick_params(labelsize=15)
ax2[0].legend(loc="lower left")

ax3.set_xlabel("Time / [h]", fontsize=15)
ax3.grid()
ax3.tick_params(labelsize=15)
ax3.set_yscale("log")


ax2[1].set_xlabel("Time / [h]", fontsize=15)
ax2[1].set_ylabel(r"$\mathrm{F_v/F_m}$ [rel. units]", fontsize=15)
ax2[1].grid()
ax2[1].tick_params(labelsize=15)
ax2[1].legend(loc="lower left")

ax4.set_xlabel("Time / [h]", fontsize=15)
ax4.grid()
ax4.tick_params(labelsize=15)
ax4.set_yscale("log")


ax[0].set_xlabel("Time / [h]", fontsize=15)
ax[0].set_ylabel(r"(B1 + B3)/Uact ", fontsize=15)
ax[0].grid()
ax[0].set_xlim(0.01)
ax[0].set_ylim(1e-7, 2.5e-7)
ax[0].tick_params(labelsize=15)
ax[0].legend()

ax[1].set_xlabel("Time / [h]", fontsize=15)
ax[1].set_ylabel(r"Quencher ", fontsize=15)
ax[1].grid()
ax[1].tick_params(labelsize=15)
ax[1].legend()

ax[2].set_xlabel("Time / [h]", fontsize=15)
ax[2].set_ylabel(r"Plastoquinone redox state", fontsize=15)
ax[2].grid()
ax[2].tick_params(labelsize=15)
ax[2].legend()

if TIFF:
    fig2.savefig(FIG_DIR + "ETprotection_time_sim.tiff", format="tiff", dpi=DPI)
else:
    fig2.savefig(FIG_DIR + "ETprotection_time_sim.png", dpi=DPI)
plt.show()

if TIFF:
    fig.savefig(FIG_DIR + "ETprotection_states_sim.tiff", format="tiff", dpi=DPI)
else:
    fig.savefig(FIG_DIR + "ETprotection_states_sim.png", dpi=DPI)
plt.show()

In [ ]:
Ua_lst = res_dict["woET"]["Ua_lst"]
FvFm = res_dict["woET"]["FvFm"]
plt.plot(1-np.array(Ua_lst)/2.5,np.array(FvFm)/FvFm[0], c="orange", label="w/o ET")

Ua_lst_ET = res_dict["ET"]["Ua_lst"]
FvFm_ET = res_dict["ET"]["FvFm"]
plt.plot(1-np.array(Ua_lst_ET)/2.5,np.array(FvFm_ET)/FvFm_ET[0], c="red", label="ET")

plt.hlines(0.5,0,1,color='k', linestyle="--")
plt.vlines(0.5,0,1,color='k',linestyle="--")

plt.xlabel("inactive PSII [rel. units]", fontsize=12)
plt.ylabel(r"$\mathrm{F_v/F_m}$ [rel. units]", fontsize=12)
plt.grid()
plt.tick_params(labelsize=12)
plt.legend()
plt.xlim(0,1)
plt.ylim(0,1)

if TIFF:
    plt.savefig(FIG_DIR + "ETprotection_sim.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "ETprotection_sim.png", dpi=DPI)
plt.show()


### Helper function

In [ ]:
# from: https://stackoverflow.com/questions/16992038/how-to-place-inline-labels-in-a-line-plot


def labelLine(line, x, label=None, align=True, reverse=False, **kwargs):
    ax = line.axes
    if reverse:
        xdata = line.get_xdata()[::-1]
        ydata = line.get_ydata()[::-1]
    else:
        xdata = line.get_xdata()
        ydata = line.get_ydata()

    if (x < xdata[0]) or (x > xdata[-1]):
        print("x label location is outside data range!")
        return

    # Find corresponding y co-ordinate and angle of the line
    ip = 1
    for i in range(len(xdata)):
        if x < xdata[i]:
            ip = i
            break

    y = ydata[ip - 1] + (ydata[ip] - ydata[ip - 1]) * (x - xdata[ip - 1]) / (
        xdata[ip] - xdata[ip - 1]
    )

    if not label:
        label = line.get_label()

    if align:
        # Compute the slope
        dx = xdata[ip] - xdata[ip - 1]
        dy = ydata[ip] - ydata[ip - 1]
        ang = degrees(atan2(dy, dx))

        # Transform to screen co-ordinates
        pt = np.array([x, y]).reshape((1, 2))
        trans_angle = ax.transData.transform_angles(np.array((ang,)), pt)[0]

    else:
        trans_angle = 0

    # Set a bunch of keyword arguments
    if "color" not in kwargs:
        kwargs["color"] = line.get_color()

    if ("horizontalalignment" not in kwargs) and ("ha" not in kwargs):
        kwargs["ha"] = "center"

    if ("verticalalignment" not in kwargs) and ("va" not in kwargs):
        kwargs["va"] = "center"

    if "backgroundcolor" not in kwargs:
        kwargs["backgroundcolor"] = ax.get_facecolor()

    if "clip_on" not in kwargs:
        kwargs["clip_on"] = True

    if "zorder" not in kwargs:
        kwargs["zorder"] = 2.5

    ax.text(x, y, label, rotation=trans_angle, **kwargs)


def labelLines(lines, align=True, xvals=None, **kwargs):
    ax = lines[0].axes
    labLines = []
    labels = []

    # Take only the lines which have labels other than the default ones
    for line in lines:
        label = line.get_label()
        if "_line" not in label:
            labLines.append(line)
            labels.append(label)

    if xvals is None:
        xmin, xmax = ax.get_xlim()
        xvals = np.linspace(xmin, xmax, len(labLines) + 2)[1:-1]

    for line, x, label in zip(labLines, xvals, labels):
        labelLine(line, x, label, align, **kwargs)



### Experiment dataclass

In [ ]:
from dataclasses import dataclass


@dataclass
class Experiment:
    id: str
    FvFm: list
    Fm: list
    Fo: list
    total_time: list
    abs_time: list
    pars: dict
    SPint: int

    @property
    def normalized_Fm(self):
        if self.Fm:
            first_number = self.Fm[0]
            return np.array(self.Fm) / first_number
        return []

    @property
    def normalized_Fo(self):
        if self.Fo:
            first_number = self.Fo[0]
            return np.array(self.Fo) / first_number
        return []

### Figure 6: Phase space

In [ ]:
# Data generation

time_prot = [20 * 60, 4 * 60 * 60]
PFD_range = range(100, 1050, 50)

if check_file_exists(RES_DIR + "phase_space.p"):
    with open(RES_DIR + "phase_space.p", "rb") as file:
        res = pickle.load(file)
else:
    PFDprot_scenarios = {"S_" + str(i): [DARK, i] for i in PFD_range}

    res = {}
    for name, PFDprot in PFDprot_scenarios.items():
        m_PS2 = deepcopy(model)
        m_PS2_ATP = deepcopy(model)

        # with quencher
        Phase_space = changingLight(m_PS2, ss_o, PFDprot, time_prot)
        compounds = Phase_space.get_full_results_df()
        flux = Phase_space.get_fluxes_df()
        time = Phase_space.get_time()

        # with quencher without dynamic ATPsynthase activation
        m_PS2_ATP.update_parameters({"activationATP_switch": 0})
        Phase_space = changingLight(m_PS2_ATP, ss_o, PFDprot, time_prot)
        compounds_ATP = Phase_space.get_full_results_df()
        flux_ATP = Phase_space.get_fluxes_df()
        time_ATP = Phase_space.get_time()

        # with base quencher only
        m_PS2.update_parameters({"kProt": 0, "kDeprot": 0, "kEpoxZ": 0, "kDeepoxV": 0})
        Phase_space = changingLight(m_PS2, ss_o, PFDprot, time_prot)
        compounds_Q = Phase_space.get_full_results_df()
        flux_Q = Phase_space.get_fluxes_df()
        time_Q = Phase_space.get_time()

        # with base quencher only and without dynamic ATPsynthase activation
        m_PS2_ATP.update_parameters(
            {"kProt": 0, "kDeprot": 0, "kEpoxZ": 0, "kDeepoxV": 0}
        )
        Phase_space = changingLight(m_PS2_ATP, ss_o, PFDprot, time_prot)
        compounds_ATP_Q = Phase_space.get_full_results_df()
        flux_ATP_Q = Phase_space.get_fluxes_df()
        time_ATP_Q = Phase_space.get_time()

        res[name] = {
            "compounds": compounds,
            "flux": flux,
            "PFD": PFDprot[0],
            "time": time,
            "compounds_Q": compounds_Q,
            "flux_Q": flux_Q,
            "PFD_Q": PFDprot[0],
            "time_Q": time_Q,
            "compounds_ATP": compounds_ATP,
            "flux_ATP": flux_ATP,
            "PFD_ATP": PFDprot[0],
            "time_ATP": time_ATP,
            "compounds_ATP_Q": compounds_ATP_Q,
            "flux_ATP_Q": flux_ATP_Q,
            "PFD_ATP_Q": PFDprot[0],
            "time_ATP_Q": time_ATP_Q,
        }

    with open(RES_DIR + "phase_space.p", "wb") as file:
        pickle.dump(res, file)

# Plot
FONTSIZE = 10
fig, ax = plt.subplots(2, 2, figsize=(18, 18))
ax = ax.flatten()
total = [1, 0.8, 0.6, 0.4, 0.2]
X = np.linspace(0, 5, 10000)

for i in total:

    def f(x):
        return 2.5 * i - x

    ax[0].plot(X, f(X), c="k", alpha=0.2)
    labelLine(
        ax[0].get_lines()[-1],
        0.35,
        label=str(int(i * 100)) + "%",
        bbox=dict(facecolor="white", edgecolor="none", pad=3),
        fontsize=FONTSIZE,
    )

    ax[1].plot(X, f(X), c="k", alpha=0.2)
    labelLine(
        ax[1].get_lines()[-1],
        0.35,
        label=str(int(i * 100)) + "%",
        bbox=dict(facecolor="white", edgecolor="none", pad=3),
        fontsize=FONTSIZE,
    )

    ax[2].plot(X, f(X), c="k", alpha=0.2)
    labelLine(
        ax[0].get_lines()[-1],
        0.35,
        label=str(int(i * 100)) + "%",
        bbox=dict(facecolor="white", edgecolor="none", pad=3),
        fontsize=FONTSIZE,
    )

    ax[3].plot(X, f(X), c="k", alpha=0.2)
    labelLine(
        ax[1].get_lines()[-1],
        0.35,
        label=str(int(i * 100)) + "%",
        bbox=dict(facecolor="white", edgecolor="none", pad=3),
        fontsize=FONTSIZE,
    )

end_points = []
Ua_points = []
dict_vPI_time = {}

end_points_Q = []
Ua_points_Q = []
dict_vPI_time_Q = {}

end_points_ATP = []
Ua_points_ATP = []
dict_vPI_time_ATP = {}

end_points_ATP_Q = []
Ua_points_ATP_Q = []
dict_vPI_time_ATP_Q = {}

for i in res:
    PFD = int(i.split("_")[-1])
    if PFD % 100 == 0:
        linestyle = "-"
        linewidth = 3
        color = "red"
        path_effects = [mpe.Stroke(linewidth=4, foreground="black"), mpe.Normal()]
    else:
        linestyle = "None"
        linewidth = 1
        color = "k"
        path_effects = [mpe.Stroke(linewidth=1, foreground="black"), mpe.Normal()]

    ax[0].plot(
        res[i]["compounds"]["B0"],
        res[i]["compounds"]["B2"],
        linewidth=linewidth,
        color=color,
        path_effects=path_effects,
        linestyle=linestyle,
    )

    ax[1].plot(
        res[i]["compounds_Q"]["B0"],
        res[i]["compounds_Q"]["B2"],
        linewidth=linewidth,
        color=color,
        path_effects=path_effects,
        linestyle=linestyle,
    )

    ax[2].plot(
        res[i]["compounds_ATP"]["B0"],
        res[i]["compounds_ATP"]["B2"],
        linewidth=linewidth,
        color=color,
        path_effects=path_effects,
        linestyle=linestyle,
    )

    ax[3].plot(
        res[i]["compounds_ATP_Q"]["B0"],
        res[i]["compounds_ATP_Q"]["B2"],
        linewidth=linewidth,
        color=color,
        path_effects=path_effects,
        linestyle=linestyle,
    )

    if PFD % 100 == 0:
        lines = ax[0].get_lines()
        labelLine(
            lines[-1],
            res[i]["compounds"]["B0"].iloc[-1] + 0.025,
            reverse=True,
            label=PFD,
            bbox=dict(facecolor="white", edgecolor="none", pad=3),
            fontsize=FONTSIZE,
        )

        lines = ax[1].get_lines()
        labelLine(
            lines[-1],
            res[i]["compounds_Q"]["B0"].iloc[-1] + 0.025,
            reverse=True,
            label=PFD,
            bbox=dict(facecolor="white", edgecolor="none", pad=3),
            fontsize=FONTSIZE,
        )

        lines = ax[2].get_lines()
        labelLine(
            lines[-1],
            res[i]["compounds_ATP"]["B0"].iloc[-1] + 0.025,
            reverse=True,
            label=PFD,
            bbox=dict(facecolor="white", edgecolor="none", pad=3),
            fontsize=FONTSIZE,
        )

        lines = ax[3].get_lines()
        labelLine(
            lines[-1],
            res[i]["compounds_ATP_Q"]["B0"].iloc[-1] + 0.025,
            reverse=True,
            label=PFD,
            bbox=dict(facecolor="white", edgecolor="none", pad=3),
            fontsize=FONTSIZE,
        )

    end_points.append(
        [res[i]["compounds"]["B0"].iloc[-1], res[i]["compounds"]["B2"].iloc[-1]]
    )
    Ua_points.append(res[i]["compounds"]["Uact"].iloc[-1])
    dict_vPI_time[i] = {
        "vPI": res[i]["flux"]["vPhotInh"],
        "time": res[i]["time"],
        "Q": res[i]["compounds"]["Q"],
    }

    end_points_Q.append(
        [res[i]["compounds_Q"]["B0"].iloc[-1], res[i]["compounds_Q"]["B2"].iloc[-1]]
    )
    Ua_points_Q.append(res[i]["compounds_Q"]["Uact"].iloc[-1])
    dict_vPI_time_Q[i] = {
        "vPI": res[i]["flux_Q"]["vPhotInh"],
        "time": res[i]["time_Q"],
        "Q": res[i]["compounds_Q"]["Q"],
    }

    end_points_ATP.append(
        [res[i]["compounds_ATP"]["B0"].iloc[-1], res[i]["compounds_ATP"]["B2"].iloc[-1]]
    )
    Ua_points_ATP.append(res[i]["compounds_ATP"]["Uact"].iloc[-1])
    dict_vPI_time_Q[i] = {
        "vPI": res[i]["flux_ATP"]["vPhotInh"],
        "time": res[i]["time_ATP"],
        "Q": res[i]["compounds_ATP"]["Q"],
    }

    end_points_ATP_Q.append(
        [
            res[i]["compounds_ATP_Q"]["B0"].iloc[-1],
            res[i]["compounds_ATP_Q"]["B2"].iloc[-1],
        ]
    )
    Ua_points_ATP_Q.append(res[i]["compounds_ATP_Q"]["Uact"].iloc[-1])
    dict_vPI_time_Q[i] = {
        "vPI": res[i]["flux_ATP_Q"]["vPhotInh"],
        "time": res[i]["time_ATP_Q"],
        "Q": res[i]["compounds_ATP_Q"]["Q"],
    }


end_points = np.array(end_points)
end_points = np.delete(end_points, 4, axis=0)
spl = make_interp_spline(end_points[:, 0][::-1], end_points[:, 1][::-1])
xx = np.linspace(end_points[:, 0][-1], end_points[:, 0][0])
ax[0].plot(
    xx, spl(xx) - 0.01, c="orange", linestyle="--", linewidth=2
)  # -0.01 just vor visualization

end_points = np.array(end_points_Q)
spl = make_interp_spline(end_points[:, 0][::-1], end_points[:, 1][::-1])
xx = np.linspace(end_points[:, 0][-1], end_points[:, 0][0])
ax[1].plot(
    xx, spl(xx) - 0.01, c="orange", linestyle="--", linewidth=2
)  # -0.01 just vor visualization

end_points = np.array(end_points_ATP)
end_points = np.delete(end_points, 4, axis=0)
spl = make_interp_spline(end_points[:, 0][::-1], end_points[:, 1][::-1])
xx = np.linspace(end_points[:, 0][-1], end_points[:, 0][0])
ax[2].plot(
    xx, spl(xx) - 0.01, c="orange", linestyle="--", linewidth=2
)  # -0.01 just vor visualization

end_points = np.array(end_points_ATP_Q)
spl = make_interp_spline(end_points[:, 0][::-1], end_points[:, 1][::-1])
xx = np.linspace(end_points[:, 0][-1], end_points[:, 0][0])
ax[3].plot(
    xx, spl(xx) - 0.01, c="orange", linestyle="--", linewidth=2
)  # -0.01 just vor visualization

ax[0].set_xlabel("open PSII [mmol / (mol Chl)]", size=20)
ax[0].set_ylabel("closed PSII [mmol / (mol Chl)]", size=20)
ax[0].tick_params(labelsize=20)
ax[0].set_xlim(0.25, 2.5)
ax[0].set_ylim(0, 2.5)

ax[1].set_xlabel("open PSII [mmol / (mol Chl)]", size=20)
ax[1].set_ylabel("closed PSII [mmol / (mol Chl)]", size=20)
ax[1].tick_params(labelsize=20)
ax[1].set_xlim(0.25, 2.5)
ax[1].set_ylim(0, 2.5)

ax[2].set_xlabel("open PSII [mmol / (mol Chl)]", size=20)
ax[2].set_ylabel("closed PSII [mmol / (mol Chl)]", size=20)
ax[2].tick_params(labelsize=20)
ax[2].set_xlim(0.25, 2.5)
ax[2].set_ylim(0, 2.5)

ax[3].set_xlabel("open PSII [mmol / (mol Chl)]", size=20)
ax[3].set_ylabel("closed PSII [mmol / (mol Chl)]", size=20)
ax[3].tick_params(labelsize=20)
ax[3].set_xlim(0.25, 2.5)
ax[3].set_ylim(0, 2.5)


left, bottom, width, height = [0.31, 0.715, 0.16, 0.16]
ax2 = fig.add_axes([left, bottom, width, height])
ax2.scatter(PFD_range, np.array(Ua_points) / 2.5 * 100, s=20)
ax2.set_ylim(bottom=60)
ax2.grid()
ax2.set_xlabel(r"PPFD ($\mathrm{µmol \cdot m^{-2} s^{-1}}$)")
ax2.set_ylabel("active PSII (%)")

left, bottom, width, height = [0.732, 0.715, 0.16, 0.16]
ax3 = fig.add_axes([left, bottom, width, height])
ax3.scatter(PFD_range, np.array(Ua_points_Q) / 2.5 * 100, s=20)
ax3.set_ylim(bottom=60)
ax3.grid()
ax3.set_xlabel(r"PPFD ($\mathrm{µmol \cdot m^{-2} s^{-1}}$)")
ax3.set_ylabel("active PSII (%)")

left, bottom, width, height = [0.31, 0.295, 0.16, 0.16]
ax3 = fig.add_axes([left, bottom, width, height])
ax3.scatter(PFD_range, np.array(Ua_points_ATP) / 2.5 * 100, s=20)
ax3.set_ylim(bottom=60)
ax3.grid()
ax3.set_xlabel(r"PPFD ($\mathrm{µmol \cdot m^{-2} s^{-1}}$)")
ax3.set_ylabel("active PSII (%)")

left, bottom, width, height = [0.732, 0.295, 0.16, 0.16]
ax4 = fig.add_axes([left, bottom, width, height])
ax4.scatter(PFD_range, np.array(Ua_points_ATP_Q) / 2.5 * 100, s=20)
ax4.set_ylim(bottom=60)
ax4.grid()
ax4.set_xlabel(r"PPFD ($\mathrm{µmol \cdot m^{-2} s^{-1}}$)")
ax4.set_ylabel("active PSII (%)")

ax[0].annotate(
    "Light on",
    xy=(0.8, 1.8),
    xytext=(2.2, 0.34),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[0].annotate(
    "",
    xy=(0.9, 1.46),
    xytext=(0.43, 1.95),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[0].text(x=0.5, y=1.5, s="Q + ATPsyn.", rotation=-42)

ax[0].annotate(
    "",
    xy=(0.86, 0.86),
    xytext=(0.9, 1.45),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[0].text(x=0.8, y=1.17, s="PI", rotation=87)

ax[1].annotate(
    "Light on",
    xy=(0.8, 1.8),
    xytext=(2.2, 0.34),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[1].annotate(
    "",
    xy=(0.75, 1.61),
    xytext=(0.42, 1.97),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[1].text(x=0.45, y=1.65, s="ATPsyn.", rotation=-40)

ax[1].annotate(
    "",
    xy=(0.7, 0.8),
    xytext=(0.75, 1.60),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[1].text(x=0.65, y=1.25, s="PI", rotation=87)

ax[2].annotate(
    "Light on",
    xy=(0.8, 1.8),
    xytext=(2.2, 0.34),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[2].annotate(
    "",
    xy=(0.9, 1.46),
    xytext=(0.73, 1.65),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[2].text(x=0.75, y=1.5, s="Q", rotation=-42)

ax[2].annotate(
    "",
    xy=(0.86, 0.86),
    xytext=(0.9, 1.45),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[2].text(x=0.8, y=1.17, s="PI", rotation=87)

ax[3].annotate(
    "Light on",
    xy=(0.8, 1.8),
    xytext=(2.2, 0.34),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[3].annotate(
    "",
    xy=(0.7, 0.8),
    xytext=(0.75, 1.60),
    size=12,
    arrowprops=dict(facecolor="grey", width=2),
)

ax[3].text(x=0.65, y=1.25, s="PI", rotation=87)


if TIFF:
    plt.savefig(FIG_DIR + "Phase_Space.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "Phase_Space.png", dpi=DPI)
plt.show()

In [ ]:
fig = plt.figure(figsize=(7, 7))

for i in dict_vPI_time:
    PFD = int(i.split("_")[-1])
    if PFD % 500 == 0:
        linewidth = 2

        plt.plot(
            dict_vPI_time[i]["time"] / 3600,
            dict_vPI_time[i]["vPI"] * 3600,
            linewidth=linewidth,
            color="red",
        )

        lines = plt.gca().get_lines()
        labelLine(
            lines[-1],
            x=0.7,
            label=PFD,
            bbox=dict(facecolor="white", edgecolor="none", pad=3),
        )

        plt.plot(
            dict_vPI_time[i]["time"] / 3600,
            dict_vPI_time[i]["Q"],
            linewidth=linewidth,
            color="k",
            linestyle="--",
        )

        # place = {500: 1.5, 1000: 3}

        # lines = plt.gca().get_lines()
        # labelLine(
        #     lines[-1],
        #     x=0.6,
        #     label=PFD,
        #     bbox=dict(facecolor="white", edgecolor="none", pad=3),
        # )


plt.xlabel("Time / [h]", fontsize=12)
plt.ylabel(r"Photoinhibition rate / [mmol / ($\mathrm{h \cdot mol~Chl}$)]", fontsize=12)
plt.grid()
plt.tick_params(labelsize=12)
plt.xlim(0, 1)
if TIFF:
    plt.savefig(FIG_DIR + "Quencher-PhaseSpace.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "Quencher-PhaseSpace.png", dpi=DPI)
plt.show()

In [ ]:
fig = plt.figure(figsize=(7, 7))

plt.plot(
    res["S_1000"]["time"] / 3600,
    res["S_1000"]["compounds"]["B0"],
    c="k",
    linewidth=2,
    linestyle="--",
    label="B0",
)
plt.plot(
    res["S_1000"]["time"] / 3600,
    res["S_1000"]["compounds"]["B2"],
    c="r",
    linewidth=2,
    linestyle="-.",
    label="B2",
)
plt.plot(
    res["S_1000"]["time"] / 3600,
    res["S_1000"]["compounds"]["P"] / model.parameters["PQtot"],
    linewidth=2,
    label="PQH2",
)
plt.title("Changes in closed and open PSII over time", fontsize=12)
plt.xlabel("Time / [h]", fontsize=12)
plt.ylabel("PSII states / [mmol/(mol Chl)] and redox state", fontsize=12)
plt.xlim(0.3, 0.7)
plt.grid()
plt.tick_params(labelsize=12)
plt.legend()

if TIFF:
    plt.savefig(FIG_DIR + "PSII-time-PhaseSpace.tiff", format="tiff", dpi=DPI)
else:
    plt.savefig(FIG_DIR + "PSII-time-PhaseSpace.png", dpi=DPI)
plt.show()

### Figure 5: Experiment

In [ ]:
df = pd.read_csv("data/experiment_norm.csv")
colw = df[(df["GT"] == "col-0") & (df["Treatment"] == "water")]
data0 = colw["meanY"]
data0.index = np.arange(0, len(data0))
sd0 = colw["sdY"]
data4 = colw["newmeanF"]
data4.index = np.arange(0, len(data4))
sd4 = colw["new_sdF"]
data8 = colw["newmeanFm"]
data8.index = np.arange(0, len(data8))
sd8 = colw["new_sdFm"]

coll = df[(df["GT"] == "col-0") & (df["Treatment"] == "5mM")]
data1 = coll["meanY"]
data1.index = np.arange(0, len(data1))
sd1 = coll["sdY"]
data5 = coll["newmeanF"]
data5.index = np.arange(0, len(data5))
sd5 = coll["new_sdF"]
data9 = coll["newmeanFm"]
data9.index = np.arange(0, len(data9))
sd9 = coll["new_sdFm"]

npqw = df[(df["GT"] == "npq-1") & (df["Treatment"] == "water")]
data3 = npqw["meanY"]
data3.index = np.arange(0, len(data3))
sd3 = npqw["sdY"]
data7 = npqw["newmeanF"]
data7.index = np.arange(0, len(data7))
sd7 = npqw["new_sdF"]
data11 = npqw["newmeanFm"]
data11.index = np.arange(0, len(data11))
sd11 = npqw["new_sdFm"]

npql = df[(df["GT"] == "npq-1") & (df["Treatment"] == "5mM")]
data2 = npql["meanY"]
data2.index = np.arange(0, len(data2))
sd2 = npql["sdY"]
data6 = npql["newmeanF"]
data6.index = np.arange(0, len(data6))
sd6 = npql["new_sdF"]
data10 = npql["newmeanFm"]
data10.index = np.arange(0, len(data10))
sd10 = npql["new_sdFm"]

##########################################################################################################
# %% Model experiment test
##########################################################################################################

Scenarios = {
    ("h2o", "wt"): {},
    ("h2o", "npq1"): {"kDeepoxV": 0},
    ("linco", "wt"): {"kREP": 0},
    ("linco", "npq1"): {"kREP": 0, "kDeepoxV": 0},
}

if check_file_exists(RES_DIR + "experiment.p"):
    with open(RES_DIR + "experiment.p", "rb") as file:
        exp_dictionary = pickle.load(file)
else:
    exp_dictionary = {}
    for k, v in Scenarios.items():
        model_exp = deepcopy(model)
        model_exp.update_parameters(v)
        if k == ("h2o", "wt"):
            (
                test_FvFm_lst,
                test_Fm_lst,
                test_Fo_lst,
                test_total_tm,
                test_absolute_tm,
                test_PAM1_lst,
            ) = FvFm_experiment2(model_exp, ss_o, SPint=SPINT, PAM_out=True)
        exp = Experiment(
            *[
                k,
                *FvFm_experiment2(model_exp, ss_o, SPint=SPINT),
                model_exp.get_parameters(),
                SPINT,
            ]
        )
        exp_dictionary[k] = exp

    with open(RES_DIR + "experiment.p", "wb") as file:
        pickle.dump(exp_dictionary, file)


def plot_experiment(exp_dictionary: dict, fname: str, residual: bool = False):
    tm = np.array([0, 0.5, 1, 3, 5, 6])

    mosaic = """
        ABC
        """

    fig = plt.figure(constrained_layout=True, figsize=(17, 5))
    ax_dict = fig.subplot_mosaic(mosaic)

    ax_dict["A"].plot(
        exp_dictionary[("h2o", "wt")].abs_time,
        exp_dictionary[("h2o", "wt")].FvFm,
        label="Water, wt",
        color="k",
    )
    ax_dict["A"].scatter(tm, data0, color="k", s=60)
    ax_dict["A"].errorbar(tm, data0, yerr=sd0, fmt="none", color="k", capsize=5)


    ax_dict["A"].plot(
        exp_dictionary[("h2o", "npq1")].abs_time,
        exp_dictionary[("h2o", "npq1")].FvFm,
        label="Water, npq1",
        color="blue",
    )
    ax_dict["A"].scatter(tm, data3, color="blue", marker="^", s=60)
    ax_dict["A"].errorbar(tm, data3, yerr=sd3, fmt="none", color="blue", capsize=5)


    ax_dict["A"].plot(
        exp_dictionary[("linco", "wt")].abs_time,
        exp_dictionary[("linco", "wt")].FvFm,
        label="Linco, wt",
        color="red",
    )
    ax_dict["A"].scatter(tm, data1, color="red", s=60)
    ax_dict["A"].errorbar(tm, data1, yerr=sd1, fmt="none", color="red", capsize=5)


    ax_dict["A"].plot(
        exp_dictionary[("linco", "npq1")].abs_time,
        exp_dictionary[("linco", "npq1")].FvFm,
        label="Linco, npq1",
        color="orange",
    )
    ax_dict["A"].scatter(tm, data2, color="orange", marker="^", s=60)
    ax_dict["A"].errorbar(tm, data2, yerr=sd2, fmt="none", color="orange", capsize=5)


    ax_dict["A"].set_xlabel("Time/[h]", size=15)
    ax_dict["A"].set_ylabel(r"$\mathrm{F_v/F_m}$", size=15)
    ax_dict["A"].grid()
    ax_dict["A"].legend(loc="lower left", fontsize=15)
    ax_dict["A"].tick_params(labelsize=15)

    ax_dict["C"].plot(
        exp_dictionary[("h2o", "wt")].abs_time,
        exp_dictionary[("h2o", "wt")].normalized_Fo,
        label="Water, wt",
        color="k",
    )
    ax_dict["C"].scatter(tm, data4, color="k", s=60)
    ax_dict["C"].errorbar(tm, data4, yerr=sd4, fmt="none", color="k", capsize=5)


    ax_dict["C"].plot(
        exp_dictionary[("h2o", "npq1")].abs_time,
        exp_dictionary[("h2o", "npq1")].normalized_Fo,
        label="Water, npq1",
        color="blue",
    )
    ax_dict["C"].scatter(tm, data7, color="blue", marker="^", s=60)
    ax_dict["C"].errorbar(tm, data7, yerr=sd7, fmt="none", color="blue", capsize=5)


    ax_dict["C"].plot(
        exp_dictionary[("linco", "wt")].abs_time,
        exp_dictionary[("linco", "wt")].normalized_Fo,
        label="Linco, wt",
        color="red",
    )
    ax_dict["C"].scatter(tm, data5, color="red", s=60)
    ax_dict["C"].errorbar(tm, data5, yerr=sd5, fmt="none", color="red", capsize=5)

    

    ax_dict["C"].plot(
        exp_dictionary[("linco", "npq1")].abs_time,
        exp_dictionary[("linco", "npq1")].normalized_Fo,
        label="linco npq1",
        color="orange",
    )
    ax_dict["C"].scatter(tm, data6, color="orange", marker="^", s=60)
    ax_dict["C"].errorbar(tm, data6, yerr=sd6, fmt="none", color="orange", capsize=5)

    

    ax_dict["C"].set_xlabel("Time/[h]", size=15)
    ax_dict["C"].set_ylabel(r"$\mathrm{F_o}$", size=15)
    ax_dict["C"].grid()
    ax_dict["C"].tick_params(labelsize=15)

    ax_dict["B"].plot(
        exp_dictionary[("h2o", "wt")].abs_time,
        exp_dictionary[("h2o", "wt")].normalized_Fm,
        label="Water, wt",
        color="k",
    )
    ax_dict["B"].scatter(tm, data8, color="k", s=60)
    ax_dict["B"].errorbar(tm, data8, yerr=sd8, fmt="none", color="k", capsize=5)



    ax_dict["B"].plot(
        exp_dictionary[("h2o", "npq1")].abs_time,
        exp_dictionary[("h2o", "npq1")].normalized_Fm,
        label="Water, npq1",
        color="blue",
    )
    ax_dict["B"].scatter(tm, data11, color="blue", marker="^", s=60)
    ax_dict["B"].errorbar(tm, data11, yerr=sd11, fmt="none", color="blue", capsize=5)



    ax_dict["B"].plot(
        exp_dictionary[("linco", "wt")].abs_time,
        exp_dictionary[("linco", "wt")].normalized_Fm,
        label="Linco, wt",
        color="red",
    )
    ax_dict["B"].scatter(tm, data9, color="red", s=60)
    ax_dict["B"].errorbar(tm, data9, yerr=sd9, fmt="none", color="red", capsize=5)

    

    ax_dict["B"].plot(
        exp_dictionary[("linco", "npq1")].abs_time,
        exp_dictionary[("linco", "npq1")].normalized_Fm,
        label="linco npq1",
        color="orange",
    )
    ax_dict["B"].scatter(tm, data10, color="orange", marker="^", s=60)
    ax_dict["B"].errorbar(tm, data10, yerr=sd10, fmt="none", color="orange", capsize=5)

    #resi12 = exp_dictionary[("linco", "npq1")].normalized_Fm - data10

    ax_dict["B"].set_xlabel("Time/[h]", size=15)
    ax_dict["B"].set_ylabel(r"$\mathrm{F_m}$", size=15)
    ax_dict["B"].grid()
    ax_dict["B"].tick_params(labelsize=15)

    if residual:
        values_to_find = [0, 0.5, 1, 3, 5, 6]
        indices = np.where(np.isin(exp_dictionary[("h2o", "wt")].abs_time, values_to_find))[0] #always the same

        resi1 = np.array(exp_dictionary[("h2o", "wt")].FvFm)[indices].flatten()/np.array(exp_dictionary[("h2o", "wt")].FvFm)[indices].flatten()[0] - data0/data0[0]
        resi2 = np.array(exp_dictionary[("h2o", "npq1")].FvFm)[indices].flatten()/np.array(exp_dictionary[("h2o", "npq1")].FvFm)[indices].flatten()[0] - data3/data3[0]
        resi3 = np.array(exp_dictionary[("linco", "wt")].FvFm)[indices].flatten()/np.array(exp_dictionary[("linco", "wt")].FvFm)[indices].flatten()[0] - data1/data1[0]
        resi4 = np.array(exp_dictionary[("linco", "npq1")].FvFm)[indices].flatten()/np.array(exp_dictionary[("linco", "npq1")].FvFm)[indices].flatten()[0] - data2/data1[0]
        resi5 = np.array(exp_dictionary[("h2o", "wt")].normalized_Fo)[indices].flatten() - data4
        resi6 = np.array(exp_dictionary[("h2o", "npq1")].normalized_Fo)[indices].flatten() - data7
        resi7 = np.array(exp_dictionary[("linco", "wt")].normalized_Fo)[indices].flatten() - data5
        resi8 = np.array(exp_dictionary[("linco", "npq1")].normalized_Fo)[indices].flatten() - data6
        resi9 = np.array(exp_dictionary[("h2o", "wt")].normalized_Fm)[indices].flatten() - data8
        resi10 = np.array(exp_dictionary[("h2o", "npq1")].normalized_Fm)[indices].flatten() - data11
        resi11 = np.array(exp_dictionary[("linco", "wt")].normalized_Fm)[indices].flatten() - data9
        resi12 = np.array(exp_dictionary[("linco", "npq1")].normalized_Fm)[indices].flatten() - data10
        residual = [resi1**2,resi2**2,resi3**2,resi4**2,resi5**2,resi6**2,resi7**2,resi8**2,resi9**2,resi10**2,resi11**2,resi12**2]
        print(data0/data0[0])
        print(np.array(exp_dictionary[("h2o", "wt")].FvFm)[indices].flatten()/np.array(exp_dictionary[("h2o", "wt")].FvFm)[indices].flatten()[0])
        print(np.sum(residual))

    if TIFF:
        plt.savefig(FIG_DIR + fname + ".tiff", format="tiff", dpi=DPI)
    else:
        plt.savefig(FIG_DIR + fname + ".png", dpi=DPI)
    plt.show()


plot_experiment(exp_dictionary, "Experiment")

### Figure S1: Firsttry

In [ ]:
Scenarios = {
    ("h2o", "wt"): {"beta": 1, "kT": 0},
    ("h2o", "npq1"): {"beta": 1, "kT": 0, "kDeepoxV": 0},
    ("linco", "wt"): {"beta": 1, "kT": 0, "kREP": 0},
    ("linco", "npq1"): {"beta": 1, "kT": 0, "kREP": 0, "kDeepoxV": 0},
}

if check_file_exists(RES_DIR + "firsttry.p"):
    with open(RES_DIR + "firsttry.p", "rb") as file:
        exp_dictionary = pickle.load(file)
else:
    exp_dictionary = {}
    for k, v in Scenarios.items():
        model_exp = deepcopy(model)
        model_exp.update_parameters(v)
        exp = Experiment(
            *[
                k,
                *FvFm_experiment2(model_exp, ss_o, SPint=SPINT),
                model_exp.get_parameters(),
                SPINT,
            ]
        )
        exp_dictionary[k] = exp

    with open(RES_DIR + "firsttry.p", "wb") as file:
        pickle.dump(exp_dictionary, file)

plot_experiment(exp_dictionary, "firsttry")

### Figure 7: PFD Quencher Photoinh rate

In [ ]:
PFD_range = np.arange(50, 1000, 50)
Q_range = np.arange(0.1, 0.8, 0.02)

if check_file_exists(RES_DIR + "3d.p"):
    with open(RES_DIR + "3d.p", "rb") as file:
        res_Q = pickle.load(file)
else:
    res_Q = {}
    for pfd in PFD_range:
        for q in Q_range:
            m_3d = deepcopy(model)
            m_3d.update_parameter("PFD", pfd)
            m_3d.add_parameter("fix_Q", q)
            m_3d.update_algebraic_module_from_args(
                module_name="Quencher",
                function=lambda p: p,
                derived_compounds=["Q"],
                args=["fix_Q"],
            )
            s = Simulator(m_3d)
            s.initialise(ss_o)
            s.simulate_to_steady_state()
            res_Q[(pfd, q)] = {
                "compounds": s.get_full_results_df(),
                "flux": s.get_fluxes_df(),
            }
    with open(RES_DIR + "3d.p", "wb") as file:
        pickle.dump(res_Q, file)

a = np.array(list(res_Q.keys()))
X = a[:, 0]
Y = a[:, 1]

aim = []
for i in res_Q:
    aim.append(res_Q[i]["flux"]["vPhotInh"].iloc[-1])

aim = np.array(aim)

x = np.reshape(X, (len(PFD_range), len(Q_range)))
y = np.reshape(Y, (len(PFD_range), len(Q_range)))
z = np.reshape(aim, (len(PFD_range), len(Q_range)))


fig = plt.figure(figsize=(7,7),constrained_layout=True)
ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(
    x,
    y,
    z * 3600,
    cmap="plasma",
    linewidth=0.025,
    antialiased=False,
    edgecolors="k",
)

ax.set_xlabel(r"PFD / [$\mathrm{\frac{µmol}{m^2~s}}$]", labelpad=10)
ax.set_ylabel("Quencher activity")
ax.set_zlabel(r"Photoinhibition rate / [$\mathrm{\frac{mmol}{h \cdot mol~Chl}}$]")

ax.view_init(elev=25, azim=135)
ax.set_box_aspect(aspect=None, zoom=0.8)


if TIFF:
    plt.savefig(FIG_DIR + "3d.tiff", format="tiff", dpi=DPI, bbox_inches="tight")
else:
    plt.savefig(FIG_DIR + "3d.png", dpi=DPI, bbox_inches="tight")
    plt.savefig(FIG_DIR + "3d.svg", format="svg", dpi=DPI, bbox_inches="tight")
plt.show()

### Long-term quenching

In [ ]:
Scenarios = {
    ("h2o", "wt"): {"beta": 1, "kT": 0, "slow_quenching":True},
    ("h2o", "npq1"): {"beta": 1, "kT": 0, "kDeepoxV": 0, "slow_quenching":True},
    ("linco", "wt"): {"beta": 1, "kT": 0, "kREP": 0, "slow_quenching":True},
    ("linco", "npq1"): {"beta": 1, "kT": 0, "kREP": 0, "kDeepoxV": 0, "slow_quenching":True},
}

if check_file_exists(RES_DIR + "slowquenching.p"):
    with open(RES_DIR + "slowquenching.p", "rb") as file:
        exp_dictionary = pickle.load(file)
else:
    exp_dictionary = {}
    for k, v in Scenarios.items():
        model_exp = deepcopy(model)
        model_exp.update_parameters(v)
        exp = Experiment(
            *[
                k,
                *FvFm_experiment2(model_exp, ss_o, SPint=SPINT),
                model_exp.get_parameters(),
                SPINT,
            ]
        )
        exp_dictionary[k] = exp

    with open(RES_DIR + "slowquenching.p", "wb") as file:
        pickle.dump(exp_dictionary, file)

plot_experiment(exp_dictionary, "slowquenching.png")

### Figure S2 and S3

In [ ]:
Scenarios = {
    ("h2o", "wt"): {"beta": 6, "kT": 0, "slow_quenching":False},
    ("h2o", "npq1"): {"beta": 6, "kT": 0, "kDeepoxV": 0, "slow_quenching":False},
    ("linco", "wt"): {"beta": 6, "kT": 0, "kREP": 0, "slow_quenching":False},
    ("linco", "npq1"): {"beta": 6, "kT": 0, "kREP": 0, "kDeepoxV": 0, "slow_quenching":False},
}

if check_file_exists(RES_DIR + "onlybeta6.p"):
    with open(RES_DIR + "onlybeta6.p", "rb") as file:
        exp_dictionary = pickle.load(file)
else:
    exp_dictionary = {}
    for k, v in Scenarios.items():
        model_exp = deepcopy(model)
        model_exp.update_parameters(v)
        exp = Experiment(
            *[
                k,
                *FvFm_experiment2(model_exp, ss_o, SPint=SPINT),
                model_exp.get_parameters(),
                SPINT,
            ]
        )
        exp_dictionary[k] = exp

    with open(RES_DIR + "onlybeta6.p", "wb") as file:
        pickle.dump(exp_dictionary, file)

plot_experiment(exp_dictionary, "onlybeta6.png")

In [ ]:
Scenarios = {
    ("h2o", "wt"): {"beta":11 , "kT": 0, "slow_quenching":False},
    ("h2o", "npq1"): {"beta": 11, "kT": 0, "kDeepoxV": 0, "slow_quenching":False},
    ("linco", "wt"): {"beta": 11, "kT": 0, "kREP": 0, "slow_quenching":False},
    ("linco", "npq1"): {"beta": 11, "kT": 0, "kREP": 0, "kDeepoxV": 0, "slow_quenching":False},
}

if check_file_exists(RES_DIR + "onlybeta11.p"):
    with open(RES_DIR + "onlybeta11.p", "rb") as file:
        exp_dictionary = pickle.load(file)
else:
    exp_dictionary = {}
    for k, v in Scenarios.items():
        model_exp = deepcopy(model)
        model_exp.update_parameters(v)
        exp = Experiment(
            *[
                k,
                *FvFm_experiment2(model_exp, ss_o, SPint=SPINT),
                model_exp.get_parameters(),
                SPINT,
            ]
        )
        exp_dictionary[k] = exp

    with open(RES_DIR + "onlybeta11.p", "wb") as file:
        pickle.dump(exp_dictionary, file)

plot_experiment(exp_dictionary, "onlybeta11.png")